# Self-Driving Car Engineer Nanodegree


## Project: **Finding Lane Lines on the Road** 
***
Student: Anton Avramov <lukav@lukav.com>


## Import Packages

In [3]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math
%matplotlib inline

## Define functions

In [269]:
def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def draw_lines(img, lines, color=[255, 0, 0], thickness=6):
    """
    This function finds the lines, avarage and extrapolate them
    """
    right_lines = []
    left_lines = []
    for line in lines:
        for x1,y1,x2,y2 in line:
            #Find the slope and push eigher in left or right list
            slope = ((y2-y1)/(x2-x1))
            if slope < -0.2:
                right_lines.append(line)
            elif slope > 0.2:
                left_lines.append(line)
    imshape = img.shape
    top = np.int(img.shape[0]*0.6)
    bottom = img.shape[0]
    # if we have found lines then we call process_lines for either left or write
    if left_lines:
        left_line = process_line(left_lines, top, bottom)
        cv2.line(img, (left_line[0], left_line[1]), (left_line[2], left_line[3]), color, thickness)

    if right_lines:
        right_line = process_line(right_lines, top, bottom)
        cv2.line(img, (right_line[0], right_line[1]), (right_line[2], right_line[3]), color, thickness)
    
def process_line(lines, top, bottom):
    """
    Process a list of lines and returns one avarage line extrapolated between top and bottom y axe
    """
    #Find and avarage
    avg = np.mean(lines, axis=0, dtype=int)
    avg = avg[0]
    # Find the slope and the intercept of the vector
    slope = ((avg[3] - avg[1]) / (avg[2] - avg[0]))
    intercept = avg[1] - (slope * avg[0])
    # calculate the x using the slope and intercept for top and bottom y
    x1 = np.int((top - intercept) / slope)
    y1 = top
    x2 = np.int((bottom - intercept) / slope)
    y2 = bottom
    return [x1, y1, x2, y2]

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

def color_mask(image):
    """Apply a color mask to get white and yellow lines"""
    # define range of white color
    lower = np.array([150,150,0])
    upper = np.array([255,255,255])
    
    # Threshold the HSV image to get only white/yellow colors
    mask = cv2.inRange(image, lower, upper)

    # Bitwise-AND mask and original image
    res = cv2.bitwise_and(image,image, mask= mask)
    
    return res

# My pipeline function
def pipeline(image):
    """The pipeline function to use all the techniques"""
    #Get only white and yellow
    masked = color_mask(image)

    #grayscale the image
    gray = grayscale(masked)
    
    #Gaussian smooth
    blur_gray = gaussian_blur(gray,5)
    
    #canny
    edges = canny(blur_gray, 200, 250)
    
    #Get only the region of interest
    imshape = image.shape
    int_height = np.int(image.shape[0]*0.6)
    int_width = np.int(image.shape[1]*0.47)
    vertices = np.array([[(0,imshape[0]),(int_width, int_height), (image.shape[1]-int_width, int_height), (imshape[1],imshape[0])]], dtype=np.int32)
    int_edges = region_of_interest(edges, vertices)

    #Find the lines
    lines_img = hough_lines(int_edges, 1, np.pi/180, 20, 20, 50)
    return weighted_img(lines_img, image)

def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below

    return pipeline(image)

## Test Images

Create directories and tries all the test images

In [271]:
import os
def ensure_dir(file_path):
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)

ensure_dir("test_images_output/")
ensure_dir("test_videos_output/")

for file in os.listdir("test_images/"):
    image = mpimg.imread('test_images/' + file)
    processed = pipeline(image)
    plt.imsave('test_images_output/' + file, processed)

## Test on Videos

In [13]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

Let's try the one with the solid white lane on the right first ...

In [263]:
white_output = 'test_videos_output/solidWhiteRight.mp4'
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/solidWhiteRight.mp4
[MoviePy] Writing video test_videos_output/solidWhiteRight.mp4



100%|█████████▉| 221/222 [00:05<00:00, 38.27it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidWhiteRight.mp4 

CPU times: user 9.29 s, sys: 324 ms, total: 9.61 s
Wall time: 6.46 s


Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [264]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [265]:
yellow_output = 'test_videos_output/solidYellowLeft.mp4'
clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/solidYellowLeft.mp4
[MoviePy] Writing video test_videos_output/solidYellowLeft.mp4


100%|█████████▉| 681/682 [00:16<00:00, 41.14it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidYellowLeft.mp4 

CPU times: user 28.4 s, sys: 784 ms, total: 29.2 s
Wall time: 17.2 s


In [266]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [267]:
challenge_output = 'test_videos_output/challenge.mp4'
clip3 = VideoFileClip('test_videos/challenge.mp4')
challenge_clip = clip3.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/challenge.mp4
[MoviePy] Writing video test_videos_output/challenge.mp4


100%|██████████| 251/251 [00:11<00:00, 21.67it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/challenge.mp4 

CPU times: user 14.5 s, sys: 488 ms, total: 14.9 s
Wall time: 12.9 s


In [268]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))